In [ ]:
import torch
import matplotlib
import numpy as np
import open3d as o3d

import os
import sys
sys.path.append(os.path.abspath(".."))
from lib.utils.image_utils import color_mapping


In [13]:
model_path = "../output/default/test/scene_ks3/models/model_it_30000.pth"
model_params, first_iter = torch.load(model_path)
(active_sh_degree, _xyz, _features_dc, 
 _features_rest, _scaling, _rotation, 
 _opacity, max_radii2D, xyz_gradient_accum, 
 denom, opt_dict, spatial_lr_scale)= model_params[0]

In [ ]:
xyz = _xyz.detach().cpu().numpy()

z = _xyz[:, 2].detach().cpu()
max_q = torch.quantile(z, 0.9)
min_q = torch.quantile(z, 0.1)
z = (z - min_q) / (max_q - min_q)
z = z.clamp(0, 1)

colormap=matplotlib.colormaps["rainbow"]
color = color_mapping(z, colormap)

# save as ply
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd.colors = o3d.utility.Vector3dVector(color.cpu().numpy().astype(np.float64))

o3d.io.write_point_cloud(f"xyz_it_{first_iter}.ply", pcd)
